### Enter full names of group members:

##### Name A:
##### Name B:

In [4]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [2]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [3]:
def dgim_algorithm(stream_path, N):
    
    # Create the buckets and initialize the timestamp
    
    
    # Loop through the entire data stream, one bit at a time
    with open(stream_path) as f:
        while True:
            bit = f.read(1)
            
            # Clause to break while loop at the end of the stream
            if not bit:
                break
              
            # To-do! update timestamp
            
            
            # To-do! implement the dgim algorithm here
            
                            
    return bucket_list, end_time_stamp

In [4]:
bucket = dgim_algorithm(stream_path, N)

In [5]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   

The updated list of timestamps buckets from DGIM algorithm: 
 [[99], [91, 96], [83, 89], [63, 75], [44], [6], [321, 446], [188], []]
The end timestamp: 99


#### 1.2. Query the Bucket 

In [6]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [43]:
def dgim_query(bucket, N, k):  
    
    # Extract the buckets and the end timestamp
    bucket_list, end_time_stamp = bucket
   
    
    # To-do! initialize the different variables
    

    # To-do! query the dgim bucket using the k parameters
    
    
    return math.ceil(one_count)

In [8]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [9]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 4
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     20.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 25
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     3.85 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 61
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     19.61 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 173
The true count of 1s in the last 300 bits: 150
The DGIM error for predicted 1s in the last 300 bits:     15.33 %
---------------------------------------------------------------
The total 1s in the last 500 bits by DGIM: 269
The true 

### 2. Bloom filters

In [5]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [6]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [7]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [20]:
def generate_hash(h, N):
    hash_list = []
    primes = [114649, 914843, 1382753]
    for j in range(h):
        hash_list.append(lambda s, j=j: sum(ord(s[i]) * primes[j]**i for i in range(len(s))) % N)
    return hash_list

In [21]:
hashes = generate_hash(h, bloom_size)

In [22]:
def create_bloom_filter(B, hashes, data):
    with data.open() as f:
        for name in f:
            for hash in hashes:
                B[hash(name)] = 1
    return B

In [23]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [24]:
bloom_array

array([0., 1., 0., ..., 1., 0., 0.])

#### 2.2. Verify usernames

In [25]:
def single_verify_username(bloom_array, hashes, new_user):
    
    # To-do! verify username and return a code of 0 or 1 (1 - username taken and 0 - username available)
    for hash in hashes:
        if (bloom_array[hash(new_user)] == 0):
            return 0
    return 1
    

In [26]:
# Feel free to test different usernames here

# new_username = "KazeemTDT4305"

new_username = "test"

In [27]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [28]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username test has been taken. Try again!


In [29]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open() as f:
        for name in f:
            hash_hit_count = 0
            for hash in hashes:
                if (bloom_array[hash(name)] == 0):
                    break
                elif (bloom_array[hash(name)] == 1):
                    hash_hit_count += 1
            total_name += 1
            if (hash_hit_count == 3):
                taken_name += 1
            
    return round(taken_name/total_name*100,2)   

In [30]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 24.07%
----------------------------------------------------------


### 3. Flajolet-Martin

In [37]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    # To-do! Define hash function h(x) = 6x + 1 mod 5
    hash_func = lambda x: (6 * x + 1) % 5

    # To-do! Iterate over the input stream and update maximum rightmost zero bit position
    for element in input_stream:
        hash_value = hash_func(element)
        binary_hash = bin(hash_value)[2:]
        rightmost_zero_position = binary_hash.rfind('0')

        if rightmost_zero_position > R:
            R = rightmost_zero_position

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate

In [38]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 2
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [40]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [41]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [28]:
def greedy_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using greedy algorithm
    
    return revenue

In [29]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 8
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 10
Trial 4 - Revenue generated: 7
Trial 5 - Revenue generated: 9
Trial 6 - Revenue generated: 8
Trial 7 - Revenue generated: 8
Trial 8 - Revenue generated: 9
Trial 9 - Revenue generated: 8
Trial 10 - Revenue generated: 8
------------------------------------------------
Average revenue generated for all trials:  8.4


#### 4.2. Balance Algorithm

In [30]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using balance algorithm
    cosine_similarity
    
    return revenue

In [31]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 9
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 10
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 9
Trial 6 - Revenue generated: 8
Trial 7 - Revenue generated: 10
Trial 8 - Revenue generated: 9
Trial 9 - Revenue generated: 9
Trial 10 - Revenue generated: 10
-------------------------------------------
Average revenue generated for all trials:  9.2


### 5. Recommender System

In [42]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [63]:
def user_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a user-user CF using cosine similarity as distance measure
    user_index, movie_index = tup_mu
    num_users, num_movies = rate_m.shape

    normalized_ratings = (rate_m - np.mean(rate_m, axis=1, keepdims=True)) / np.std(rate_m, axis=1, keepdims=True)
    # Use np.dot to calculate cosine similarity
    sim = np.dot(normalized_ratings, normalized_ratings.T)
    print(sim)
    # Ensure the diagonal is 1 (self-similarity)
    norms = np.sqrt(np.diagonal(sim))
    print(norms)
    sim /= np.outer(norms, norms)
    np.fill_diagonal(sim, 1) 
    

    similar_indices = np.argsort(-sim[user_index])[:neigh+1]
    if user_index in similar_indices:
        similar_indices = similar_indices[similar_indices != user_index]
    else:
        similar_indices = similar_indices[:-1]

    # Step 3: Predict the rating using weighted average of ratings by similar users
    similar_users_similarities = sim[user_index, similar_indices]
    similar_users_ratings = rate_m[similar_indices, movie_index]
    
    # Avoid division by zero by adding a small number to the denominator
    prediction = np.dot(similar_users_similarities, similar_users_ratings) / (np.sum(similar_users_similarities) + 1e-10)
    
    
    return prediction   

In [53]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [64]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
[[76 19 42 20 30 26]
 [19 71 27 22 49 19]
 [42 27 84 28 21 28]
 [20 22 28 65 40 43]
 [30 49 21 40 74 32]
 [26 19 28 43 32 39]]
[8.71779789 8.42614977 9.16515139 8.06225775 8.60232527 6.244998  ]


UFuncTypeError: Cannot cast ufunc 'divide' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'

#### 5.2. Item-Item Collaborative Filtering

In [36]:
def item_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a item-item CF using cosine similarity as distance measure
    
    return prediction

In [37]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 2.48 (Item-Item CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

In [38]:
# Enter answer here

#### Case 2

In [39]:
# Enter answer here

#### Case 3

In [40]:
# Enter answer here

#### Case 4

In [41]:
# Enter answer here

#### Case 5

In [42]:
# Enter answer here